In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import plotly
plotly.tools.set_credentials_file(username='ninjamouse', api_key='LejqbK7NWhPU7zXoMuhw')

import numpy as np
import pandas as pd

df1 = pd.read_csv('09_attack_delta_time.csv')

In [2]:
df2 = pd.read_csv("18_attack_delta_time.csv")

In [3]:
df_concat = pd.concat([df1, df2])

In [4]:
#df.sort_values(['Source', 'Destination'],  ascending=[True, False]  )
#df['Info'] = df['Info'].astype('str')
#get=df[df['Info'].str.contains('GET')]


#Sort the connections based on the pair Source-->Destination
#Select all the connections that have Source IP and Destination IP with 75.127.97.72
df_concat.drop(['Time','Delta time','Source Port','Destination Port', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()

No.
Source        Destination        
192.168.56.1  74.55.1.4      6066
              69.84.133.138  5927
74.55.1.4     192.168.56.1   2378
69.84.133.138 192.168.56.1   2239
68.178.178.97 192.168.2.112   796

In [5]:
filter_ip = ['69.84.133.138','74.55.1.4' ]
#d_concat=df_concat[(df_concat.Destination=='69.84.133.138') or (df_concat.Destination=='74.55.1.4')]
#
d_concat =df_concat[df_concat.Destination.isin(filter_ip)]
d_concat.drop(['Time','Delta time','Source Port','Destination Port', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()

No.
Source        Destination        
192.168.56.1  74.55.1.4      6066
              69.84.133.138  5927
192.168.2.110 69.84.133.138    52
192.168.1.101 74.55.1.4        23

In [6]:
#convert Info into string
#filter packets with TCP SYN
d_concat['Info'] = d_concat['Info'].astype('str')
d_concat=d_concat[d_concat['Info'].str.contains('SYN')]

In [7]:
#Display packets with IP filtered
trace1=go.Scatter(x=df_concat['Time'],y=df_concat['Length'],mode='markers', name='Original dataframe')
#l=df.query('(Length>10000)')
trace2=go.Scatter(x=d_concat['Time'],y=d_concat['Length']*10, mode='markers', name='Filtered by IP and SYN')
data=[trace1, trace2]
py.iplot(data, filename='trace 1')

The draw time for this plot will be slow for clients without much RAM.


/home/deeplearning/anaconda3/lib/python3.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow

/home/deeplearning/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [8]:
from sklearn import linear_model

In [9]:
X = df_concat

# Treat the problem as a pure classification problem.

X['Protocol'] = X['Protocol'].astype('category')
# Could use float, but it doesn't make much sense
#X['Source'] = X['Source'].apply(lambda x: float(x.replace('.','')))

# Convert IPs to classes
X['Destination'] = X['Destination'].astype('category')
X['Source'] = X['Source'].astype('category')
X['Source Port'] = X['Source Port'].astype('category')
X['Destination Port'] = X['Destination Port'].astype('category')
# Convert info to classes as well
X['Info'] = X['Info'].astype('category')  # ToDo split info in relevant columns
#X['Destination Port'] = X['Destination Port'].astype('64bit')
# Drop the time series for now, just to play with a classifier
X = X.drop(['Time', 'Delta time'], axis=1)
X.head()

,No.,Source,Source Port,Destination,Protocol,Destination Port,Length,Info
0,1,192.168.2.106,3595.0,205.234.175.175,TCP,80.0,60,"3595 > 80 [FIN, ACK] Seq=1 Ack=1 Win=17520 L..."
1,2,192.168.1.104,17053.0,192.150.8.118,TCP,80.0,60,"17053 > 80 [FIN, ACK] Seq=1 Ack=1 Win=17520 ..."
2,3,192.168.1.104,17057.0,192.150.18.200,TCP,80.0,60,"17057 > 80 [FIN, ACK] Seq=1 Ack=1 Win=16298 ..."
3,4,192.168.2.113,1302.0,63.245.209.72,TCP,443.0,60,"1302 > 443 [FIN, ACK] Seq=1 Ack=1 Win=64059 ..."
4,5,192.168.2.110,2247.0,72.21.91.19,TCP,80.0,60,"2247 > 80 [FIN, ACK] Seq=1 Ack=1 Win=63040 L..."


In [10]:
# Set some entries as 1 'attack' and 0 no attack
# The ones you defined in d above, are entries where an attack was plausible. Everything else is clean
Y = [0 if i not in d_concat.index.values else 1 for i in df_concat.index.values]

In [11]:
#Verify Y in not all zeros(0)
# for i in Y:
#     if i!=0:
#        print(i)

In [12]:
#1. Aici ce facem? Pentru fiecare valoare este asignata nu cod?
## Transformam toate coloanele care nu sunt numerice, in valori numerice.
## De exemplu, pentru coloana Protocol ai valori 'HTTP' si 'TCP'. Matematic e greu sa le analizezi
## daca nu sunt in format numeric. Asa ca functia lambda de mai jos iti transforma
## toate valorile 'HTTP' in 0 si 'TCP' in 1 (de exemplu). Daca ai mai multe coloane, ai mai multe valori (0,1,2,3..)
catColumns = X.select_dtypes(['category']).columns
X[catColumns] = X[catColumns].apply(lambda x: x.cat.codes)
X.head()

,No.,Source,Source Port,Destination,Protocol,Destination Port,Length,Info
0,1,26,227,63,19,6,60,3493
1,2,24,381,19,19,6,60,913
2,3,24,382,17,19,6,60,914
3,4,33,38,99,19,12,60,588
4,5,30,118,136,19,6,60,2207


In [13]:
#2. Banuiesc ca aici este aplicat algoritmul de linear regression? Ar trebui sa il inteleg sau doar il aplic si gata?
## Aici faci doar partea de model fitting a algoritmului. Pe baza valorilor din X, iti antrenezi modelul ca sa prezica
## valorile din Y. Aici calculezi coeficientii cei mai potriviti din formula liniara care iti satisfac cel mai bine
## X = Y
## sau mai in detaliu, ai avea 
## coef_Source * X['Source'] + coef_Destination * X['Destination'] + .. + coef_Info * X['Info'] = Y
## ,pentru fiecare linie din X si linia corespunzatoare din Y

clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X, Y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
# ## TODO

# df = pd.read_csv('second_attack.csv')

# X_test = df

# ## apply transformation methods as above

# pred = clf.predict(X_test)

df3 = pd.read_csv('19_attack_delta_time.csv')
df3.head()

,No.,Time,Delta time,Source,Source Port,Destination,Protocol,Destination Port,Length,Info
0,1,"2010-06-12 19:46:00,031663",0.000000,62.48.76.70,80.0,192.168.2.110,TCP,3338.0,402,"80 > 3338 [PSH, ACK] Seq=1 Ack=1 Win=49680 L..."
1,2,"2010-06-12 19:46:00,042818",0.011155,62.48.76.70,80.0,192.168.2.110,TCP,3338.0,1434,80 > 3338 [ACK] Seq=349 Ack=1 Win=49680 Len=...
2,3,"2010-06-12 19:46:00,044008",0.001190,192.168.2.110,3338.0,62.48.76.70,TCP,80.0,60,3338 > 80 [ACK] Seq=1 Ack=1729 Win=64860 Len=0
3,4,"2010-06-12 19:46:00,046592",0.002584,62.48.76.70,80.0,192.168.2.110,TCP,3338.0,1434,80 > 3338 [ACK] Seq=1729 Ack=1 Win=49680 Len...
4,5,"2010-06-12 19:46:00,050334",0.003742,62.48.76.70,80.0,192.168.2.110,TCP,3338.0,1434,80 > 3338 [ACK] Seq=3109 Ack=1 Win=49680 Len...


In [15]:
df4 = pd.read_csv('25_attack_delta_time.csv')
df4.head()

,No.,Time,Delta time,Source,Source Port,Destination,Protocol,Destination Port,Length,Info
0,1,"2010-06-13 01:26:00,332989",0.000000,69.192.24.88,80.0,192.168.56.1,TCP,55216.0,2962,80 > 55216 [ACK] Seq=1 Ack=1 Win=122 Len=289...
1,2,"2010-06-13 01:26:00,333062",0.000073,192.168.56.1,55216.0,69.192.24.88,TCP,80.0,60,55216 > 80 [RST] Seq=1 Win=0 Len=0
2,3,"2010-06-13 01:26:00,333062",0.000000,69.192.24.88,80.0,192.168.56.1,HTTP,55216.0,1294,HTTP/1.1 500 Service unavailable (with message...
3,4,"2010-06-13 01:26:00,333123",0.000061,192.168.56.1,55216.0,69.192.24.88,TCP,80.0,60,55216 > 80 [RST] Seq=1 Win=0 Len=0
4,5,"2010-06-13 01:26:00,337329",0.004206,192.168.56.1,55357.0,69.192.24.88,HTTP,80.0,599,GET /drupal7/?uIhD=PU3cMTarBFX&MUSc2Ei=1xcwRr4...


In [16]:
df3.drop(['Time', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()

,,No.,Delta time,Source Port,Destination Port
Source,Destination,,,,
192.168.56.1,69.192.24.88,6066,6066,6066,6066
69.192.24.88,192.168.56.1,2378,2378,2378,2378
203.73.24.75,192.168.2.109,1874,1874,1874,1874
192.168.2.109,203.73.24.75,1695,1695,1695,1695
85.232.57.10,192.168.1.104,254,254,254,254


In [17]:
df4.drop(['Time', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()

,,No.,Delta time,Source Port,Destination Port
Source,Destination,,,,
192.168.56.1,75.127.97.72,6066,6066,6066,6066
75.127.97.72,192.168.56.1,2378,2378,2378,2378
192.168.5.122,80.66.211.179,997,997,997,997
80.66.211.179,192.168.5.122,803,803,803,803
192.168.56.1,69.192.24.88,471,471,471,471


In [18]:
df2_concat = pd.concat([df3, df4])
filter_ip2 = ['69.192.24.88','75.127.97.72' ]
#d_concat=df_concat[(df_concat.Destination=='69.84.133.138') or (df_concat.Destination=='74.55.1.4')]
#
d2_concat =df2_concat[df2_concat.Destination.isin(filter_ip2)]

In [19]:
#d2=df2[(df2.Destination=='74.55.1.4')] 
#convert Info into string
#filter packets with TCP SYN
d2_concat['Info'] = d2_concat['Info'].astype('str')
d2_concat=d2_concat[d2_concat['Info'].str.contains('SYN')]

In [20]:
X_test=df2_concat
X_test['Protocol'] = X_test['Protocol'].astype('category')
X_test['Destination'] = X_test['Destination'].astype('category')
X_test['Source'] = X_test['Source'].astype('category')
X_test['Info'] = X_test['Info'].astype('category')  # ToDo split info in relevant columns
X_test['Source Port'] = X_test['Source Port'].astype('category')
X_test['Destination Port'] = X_test['Destination Port'].astype('category')
X_test = X_test.drop(['Time', 'Delta time'], axis=1)
X_test.head()
catColumns = X_test.select_dtypes(['category']).columns
X_test[catColumns] = X_test[catColumns].apply(lambda x: x.cat.codes)
pred = clf.predict(X_test)

In [21]:
# for i in pred:
#     if i!=0:
#         print(i)

In [22]:
trace3=go.Scatter(x=df2_concat['Time'],y=df2_concat['Length'],mode='markers', name='Original file')
#l=df2.query('(Length>10000)')
trace4=go.Scatter(x=d2_concat['Time'],y=d2_concat['Length']*10,mode='markers', name='Filtered by IP')
trace5=go.Scatter(x=df2_concat['Time'],y=pred*5000, mode='markers', name='Prediction')
data2=[trace3, trace4, trace5]
py.iplot(data2, filename='trace 2')

The draw time for this plot will be slow for all clients.


/home/deeplearning/anaconda3/lib/python3.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long

/home/deeplearning/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [23]:
## TODO
## Compute 'd' the same way you computed it in the first case
## Plot d + pred
## Compare overalp between d and pred

In [24]:
#X

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
clf.feature_importances_

array([0.14574966, 0.03352516, 0.3366047 , 0.1156106 , 0.00389039,
       0.05256261, 0.10361822, 0.20843866])

In [27]:
pred2=clf.predict(X_test)

In [28]:
# for i in pred2:
#     if i!=0:
#         print(i)

In [29]:
trace3=go.Scatter(x=df2_concat['Time'],y=df2_concat['Length'],mode='markers', name='Original file')
#l=df2.query('(Length>10000)')
trace4=go.Scatter(x=d2_concat['Time'],y=d2_concat['Length']*10,mode='markers', name='Filtered by IP and SYN')
trace5=go.Scatter(x=df2_concat['Time'],y=pred*5000, mode='markers', name='LinearModel')
trace6=go.Scatter(x=df2_concat['Time'],y=pred2*6000, mode='markers', name='Random Forest')
data2=[trace3, trace4, trace5, trace6] 
py.iplot(data2, filename='trace 3')

The draw time for this plot will be slow for all clients.


/home/deeplearning/anaconda3/lib/python3.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long

/home/deeplearning/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [30]:
test_df=df2.head()
test_df

,No.,Time,Delta time,Source,Source Port,Destination,Protocol,Destination Port,Length,Info
0,1,"2010-06-12 19:30:00,865985",0.000000,192.168.2.113,2950.0,216.8.179.23,TCP,80.0,60,"2950 > 80 [FIN, ACK] Seq=1 Ack=1 Win=64860 L..."
1,2,"2010-06-12 19:30:04,632293",3.766308,192.168.2.113,137.0,192.168.2.255,NBNS,137.0,92,Name query NB WORKGROUP<1b>
2,3,"2010-06-12 19:30:05,381433",0.749140,192.168.2.113,137.0,192.168.2.255,NBNS,137.0,92,Name query NB WORKGROUP<1b>
3,4,"2010-06-12 19:30:06,131413",0.749980,192.168.2.113,137.0,192.168.2.255,NBNS,137.0,92,Name query NB WORKGROUP<1b>
4,5,"2010-06-12 19:30:10,254751",4.123338,192.168.5.122,143.0,192.168.3.116,IMAP,2756.0,71,Response: * OK Still here


In [31]:
#test_df['Time']-test_df['Time'].shift()